In [53]:
# Check for existing libraries in environment
! pip show rdflib lxml

/bin/bash: line 1: pip: command not found


In [54]:
from lxml import etree
import os

# this set up globals for some useful URIs
globals = {
    "xml_id":"{http://www.w3.org/XML/1998/namespace}id",
    "base_URI": "https://github.com/falaimo99/sam/data/",
    }

# this function extract the tree from the well formed xml
def extract_tree(path):
    tree = etree.parse(path)
    root = tree.getroot()  
    return root

root = extract_tree("/home/owly/projects/tesi/shortStories/Novellino_II.xml")


In [ ]:
from rdflib import Graph, Literal, BNode, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS, OWL

# Namespaces used by SAM
sam = Namespace("https://purl.org/samcore#")
wdt = Namespace("https://www.wikidata.org/wiki/")
wdp = Namespace("https://www.wikidata.org/wiki/Property:")
cwrc = Namespace("https://sparql.cwrc.ca/ontologies/cwrc.html#")

# Graph Instantiation
def setting_the_graph():
    g = Graph()

    g.bind("sam", sam)
    g.bind("wdt", wdt)
    g.bind("wdp", wdp)
    g.bind("cwrc", cwrc)
    
    return g
g: Graph = setting_the_graph()

# Function to set up the story element and the mandatory related items
# CharacterList and GroupOfScenes/Scenes
def set_base_story ():
    
    story_URI = URIRef(
        globals["base_URI"] + root.find("Story").attrib[globals['xml_id']]
        )
    characterList_URI = URIRef(
        story_URI + "_CharacterList"
    )

    if root.findall('SequenceOfScenes'):
        for SoS in root.findall('SequenceOfScenes'):
            pass

    g.add ( (story_URI, RDF.type, sam.Story) )
    g.add ( (characterList_URI, RDF.type, sam.CharacterList) )
    # g.add ( (story_URI, ) )


    print(f"{story_URI} added as sam:Story;\n{characterList_URI} added as sam:CharacterList")

    return g

set_base_story()

ValueError: not enough values to unpack (expected 3, got 1)